# Backtest Short Butterfly With Stop-Loss and Take-Profit

In the previous video we saw how we can use stop-loss (SL) and take-profit (TP) as risk management tools to improve the returns and most importantly, reduce the risk. In this notebook, we will backtest the short butterfly which exits when either SL/TP conditions are met or at expiry.

The notebook is structured as follows:
1. [Import the Data](#import)
2. [Strategy Parameters](#parameters)
2. [Calculate ADX, IVP and Days To Expiry](#indicator)
3. [Entry conditions](#entry)
4. [Exit conditions](#exit)
5. [Backtesting](#backtesting)

In [1]:
# Data manipulation
import pandas as pd
import numpy as np

# Technical analysis
import talib as ta

# Datetime manipulation
from datetime import timedelta

# Ignore warnings
import warnings
warnings.simplefilter('ignore')

# Helper functions
import sys
sys.path.append('..')
from data_modules.options_util_quantra import get_IV_percentile, get_premium, get_expected_profit_empirical, setup_butterfly

<a id='import'></a>
## Import the Data

Import the files `nifty_options_data_2019_2022.bz2` and `nifty_data_2019_2022.bz2` using the `read_pickle` method of `pandas`.
These bz2 files are available in the zip file of the unit 'Python Codes and Data' in the 'Course Summary' section.

In [2]:
# Read data
options_data = pd.read_pickle('../data_modules/nifty_options_data_2019_2022.bz2')
data = pd.read_pickle('../data_modules/nifty_data_2019_2022.bz2')
data.head()

,spot_open,spot_high,spot_low,spot_close,Expiry,futures_close
Date,,,,,,
2019-01-01,10881.70,10923.60,10807.10,10910.10,2019-01-31,10960.55
2019-01-02,10868.85,10895.35,10735.05,10792.50,2019-01-31,10830.85
2019-01-03,10796.80,10814.05,10661.25,10672.25,2019-01-31,10718.50
2019-01-04,10699.70,10741.05,10628.65,10727.35,2019-01-31,10777.60
2019-01-07,10804.85,10835.95,10750.15,10771.80,2019-01-31,10803.45


<a id='parameters'></a>
## Strategy Parameters

We will set the stop-loss and take-profit percentage to 30% of the net entry premium. You can try changing this to see how it affects the backtest results. However, if the SL is kept too low, then it might get hit too frequently. On the other hand, if we keep it too high, then it might not hit at all. 

The `days_to_exit_before_expiry` can be changed if you want to exit the trade a few days before expiry and don't want to hold it till expiry to avoid huge MTM swing when expiry is near.

In [10]:
config = {
    'stop_loss_percentage': 0.3,
    'take_profit_percentage': 0.3, 
    'days_to_exit_before_expiry': 0
}

<a id='indicator'></a>
## Calculate ADX, IVP and Days To Expiry

Our entry conditions are based on ADX indicator and IVP. We will go against the technical indicator and use ADX in reverse fashion. We will create two columns: ADX and IVP in `data` to store the ADX and IVP values.

We will use ADX function from talib library to calculate ADX. It takes spot_high, spot_low, spot_close, timeperiod as input and returns the ADX value.

IVP is calculated using get_IV_percentile function which takes data, options_data, window as input and returns IVP value. This function is present in options_util_quantra.py file.


In [5]:
# Calculate ADX
data['ADX'] = ta.ADX(data.spot_high, data.spot_low, data.spot_close, timeperiod=14)

# Calculate IVP
data['IVP'] = get_IV_percentile(data, options_data, window = 60)

# Calculate days to expiry
data['days_to_expiry'] = (data['Expiry'] - data.index).dt.days

<a id='entry'></a>
## Entry Conditions

We will check the following two conditions for entry:
1. IVP should be between 50 and 95.
2. ADX value should be less than 30.

Create a column `signal_adx_ivp` which stores `1` when the above two conditions are met.

In [6]:
# IVP entry condition
condition_1 = (data['IVP'] >= 50) & (data['IVP'] <= 95)

# ADX entry condition
condition_2 = (data['ADX'] <= 30)

# Generate signal as 1 when both conditions are true
data['signal_adx_ivp'] = np.where(condition_1 & condition_2, 1, np.nan)

<a id='exit'></a>
## Exit Conditions

`signal_adx_ivp` column stores `0` when `days_to_expiry` is less than `days_to_exit_before_expiry`.

We will exit the trade if the net_premium on the given date crosses above the take-profit, below the stop-loss or at expiry. The conditions for SL and TP have been defined in the next part of the code.

In [7]:
# Generate signal as 0 when days to expiry is less than days to exit before expiry
data['signal_adx_ivp'] = np.where(data.days_to_expiry <= config['days_to_exit_before_expiry'], 0, data['signal_adx_ivp'])
data.tail()

,spot_open,spot_high,spot_low,spot_close,Expiry,futures_close,ADX,atm_strike_price,IV,IVP,days_to_expiry,signal_adx_ivp
Date,,,,,,,,,,,,
2022-05-20,16043.80,16283.05,16003.85,16266.15,2022-05-26,16253.25,25.945619,16250.0,21.617889,53.333333,6,1.0
2022-05-23,16290.95,16414.70,16185.75,16214.70,2022-05-26,16183.35,24.958370,16200.0,26.973724,93.333333,3,1.0
2022-05-24,16225.55,16262.80,16078.60,16125.15,2022-05-26,16104.70,24.320691,16100.0,26.496887,90.000000,2,1.0
2022-05-25,16196.35,16223.35,16006.95,16025.80,2022-05-26,16013.80,23.914622,16000.0,28.594971,100.000000,1,NaN
2022-05-26,16105.00,16204.45,15903.70,16170.15,2022-05-26,16159.05,23.804497,16150.0,0.000000,99.166667,0,0.0


<a id='backtesting'></a>
## Backtesting

We will loop over each of the dates in the data, setup the butterfly when entry conditions are met, exit when exit conditions are met, update the trade in round trips. mark_to_market dataframe contains the premiums of the strategy on each date between the entry date and exit date.

We will backtest the butterfly strategy using the following steps:

**Step-1**: Create dataframes `round_trips_details`, `trades` and `mark_to_market` for storing round trips, trades and mtm.

**Step-2**: Define a function `add_to_mtm` which stores daily mark_to_market values for the strategy. It takes existing `mark_to_market` dataframe, `option_strategy` which is butterfly in this case and `trading_date` as inputs.

**Step-3**: Initialise `current_position`, `trade_num` which is basically number of trades, `cum_pnl` to 0 and set the `entry_flag` to `False`. 

**Step-4**: We also set the `start_date` for backtesting. Expected profit requires 90 days of historical data. So we will keep the `start_ date` accordingly. 

In [ ]:
# Create dataframes for round trips, storing trades, and mtm
round_trips_details = pd.DataFrame()
trades = pd.DataFrame()
mark_to_market = pd.DataFrame()

# Function for calculating mtm
def add_to_mtm(mark_to_market, option_strategy, trading_date):
    option_strategy['Date'] = trading_date
    mark_to_market = pd.concat([mark_to_market, option_strategy])
    return mark_to_market

# Initialise current position, number of trades,cumulative pnl, stop-loss and take-profit to None
current_position = 0
trade_num = 0
cum_pnl = 0
sl = None
tp = None

# Set exit flag to False
exit_flag = False

# Set start date for backtesting
start_date = data.index[0] + timedelta(days=90)

Perform the following steps iteratively for the dates in the backtest period.

**Step-5**: For a given date, if there is no open position and entry conditions are met we will setup the butterfly and calculate the expected profit for the butterfly.

**Step-6**: Here expected profit is calculated using empirical distribution which uses historical data points for calculating the probability. We will take a trade here with a view that is against the historical data i.e. when expected profit is less than 0.

**Step-7**: For a given date, if there is an open position we exit the trade if stop-loss/take-profit gets hit or if the given date is an expiry and update round trips.

**Step-8**: Finally we calculate the pnl for the trade and also the cumulative pnl.

In [ ]:
for i in data.loc[start_date:].index:

    if (current_position == 0) & (data.loc[i, 'signal_adx_ivp'] == 1):
        
        options_data_daily = options_data.loc[i]
        
        # Setup butterfly
        butterfly = setup_butterfly(data.loc[i,'futures_close'], options_data_daily, direction = "short") 
                   
        # Calculate Expected profit        
        price_range = list(options_data_daily['Strike Price'].unique())        
        
        from datetime import timedelta
        start_date_ep = i - timedelta(days=90)        
        
        data.loc[i,'exp_profit'] = get_expected_profit_empirical(data.loc[start_date_ep:i], 
                                    butterfly.copy(), data.loc[i, 'days_to_expiry'], price_range)
        
        if data.loc[i,'exp_profit'] < 0:
            
            # Check that the last price of any of the leg of the butterfly should be greater than 0
            if (butterfly.premium.isna().sum() > 0) or ((butterfly.premium == 0).sum() > 0):
                print(f"\x1b[31mStrike price not liquid so we will ignore this trading opportunity {i}\x1b[0m")
                continue
            
            # Populate the trades dataframe
            trades = butterfly.copy()
            trades['entry_date'] = i
            trades.rename(columns={'premium':'entry_price'}, inplace=True)            
            
            # Calculate net premium 
            net_premium = round((butterfly.position * butterfly.premium).sum(),1)
            
            # Compute SL and TP for the trade
            sl = net_premium * (1 - config['stop_loss_percentage'])
            tp = net_premium * (1 + config['take_profit_percentage'])
            
            # Update current position to 1
            current_position = 1
            
            # Update mark_to_market dataframe
            mark_to_market = add_to_mtm(mark_to_market, butterfly, i)
            
            # Increase number of trades by 1
            trade_num += 1   
            print("-"*30)
            
            # Print trade details
            print(f"Trade No: {trade_num} | Entry | Date: {i} | Premium: {net_premium} | Pnl: 0 | Cum PnL: {cum_pnl}")            
            
    elif current_position == 1:
        
        # Update net premium
        options_data_daily = options_data.loc[i]
        butterfly['premium'] = butterfly.apply(lambda r: get_premium(r, options_data_daily), axis=1)        
        net_premium = (butterfly.position * butterfly.premium).sum()
        
        # Update mark_to_market dataframe
        mark_to_market = add_to_mtm(mark_to_market, butterfly, i)
     
        # Exit the trade if any of the exit condition is met
        if data.loc[i, 'signal_adx_ivp'] == 0:
            exit_type = 'Expiry'
            exit_flag = True
                   
        elif net_premium < sl:
            exit_type = 'SL'
            exit_flag = True
                           
        elif net_premium > tp:                               
            exit_type = 'TP'
            exit_flag = True
            
        if exit_flag:
            
            # Check that the data is present for all strike prices on the exit date
            if butterfly.premium.isna().sum() > 0:
                print(f"Data missing for the required strike prices on {i}, Not adding to trade logs.")
                current_position = 0
                exit_flag = False
                continue
            
            # Append the trades dataframe
            trades['exit_date'] = i
            trades['exit_type'] = exit_type
            trades['exit_price'] = butterfly.premium
            
            # Add the trade logs to round trip details
            round_trips_details = pd.concat([round_trips_details,trades])
            
            # Calculate net premium at exit
            net_premium = round((butterfly.position * butterfly.premium).sum(),1)   
            
            # Calculate net premium on entry
            entry_net_premium = (trades.position * trades.entry_price).sum()       
            
            # Calculate pnl for the trade
            trade_pnl = round(net_premium - entry_net_premium,1)
            
            # Calculate cumulative pnl
            cum_pnl += trade_pnl
            cum_pnl = round(cum_pnl,2)
            
            # Print trade details
            print(f"Trade No: {trade_num} | Exit Type: {exit_type} | Date: {i} | Premium: {net_premium} | PnL: {trade_pnl} | Cum PnL: {cum_pnl}")                              

            # Update current position to 0
            current_position = 0    
            
            # Set exit flag to false
            exit_flag = False                

Strike price not liquid so we will ignore this trading opportunity 2019-05-09 00:00:00
------------------------------
Trade No: 1 | Entry | Date: 2019-06-24 00:00:00 | Premium: 94.8 | Pnl: 0 | Cum PnL: 0
Trade No: 1 | Exit Type: TP | Date: 2019-06-26 00:00:00 | Premium: 125.4 | PnL: 30.7 | Cum PnL: 30.7
------------------------------
Trade No: 2 | Entry | Date: 2019-06-28 00:00:00 | Premium: 241.4 | Pnl: 0 | Cum PnL: 30.7
Trade No: 2 | Exit Type: TP | Date: 2019-07-22 00:00:00 | Premium: 342.8 | PnL: 101.4 | Cum PnL: 132.1
------------------------------
Trade No: 3 | Entry | Date: 2019-09-11 00:00:00 | Premium: 173.6 | Pnl: 0 | Cum PnL: 132.1
Trade No: 3 | Exit Type: TP | Date: 2019-09-23 00:00:00 | Premium: 292.7 | PnL: 119.0 | Cum PnL: 251.1
------------------------------
Trade No: 4 | Entry | Date: 2019-10-04 00:00:00 | Premium: 279.6 | Pnl: 0 | Cum PnL: 251.1
Trade No: 4 | Exit Type: TP | Date: 2019-10-29 00:00:00 | Premium: 394.2 | PnL: 114.6 | Cum PnL: 365.7
---------------------

In [9]:
# Round trip details
round_trips_details.head()

,Option Type,Strike Price,position,entry_price,entry_date,exit_date,exit_type,exit_price
0,CE,11700,1,73.70,2019-06-24,2019-06-26,TP,161.55
1,PE,11700,1,54.00,2019-06-24,2019-06-26,TP,4.70
2,CE,11850,-1,17.95,2019-06-24,2019-06-26,TP,39.25
3,PE,11550,-1,15.00,2019-06-24,2019-06-26,TP,1.60
0,CE,11850,1,174.00,2019-06-28,2019-07-22,TP,1.15


In [10]:
# MTM details (We will use this dataframe in upcoming notebooks for trade level analytics)
mark_to_market.head()

,Option Type,Strike Price,position,premium,Date
0,CE,11700,1,73.70,2019-06-24
1,PE,11700,1,54.00,2019-06-24
2,CE,11850,-1,17.95,2019-06-24
3,PE,11550,-1,15.00,2019-06-24
0,CE,11700,1,106.00,2019-06-25


## Conclusion

In this notebook we backtested a short butterfly strategy which takes trade based on IVP and ADX values and exits the same based on stop-loss and take-profit or on expiry. On comparing both the strategies we can see that the strategy with SL and TP as exit conditions performs better.